In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from sklearn import preprocessing
import os  

torch.manual_seed(1)
np.random.seed(1)
random.seed(1)


class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()
    
    def forward(self, output, target):
        eps = 1e-8
        loss = torch.mean(torch.abs((target - output) / (target + eps)))
        return loss


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class FeatureDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.x[idx]), torch.FloatTensor(self.y[idx])

def normalizing_data(data, seed=1, save_path='normalized_data_excel'):  
    os.makedirs(save_path, exist_ok=True)
    
    comp_cols = ['Ba', 'Ca', 'Sr', 'Ti', 'Zr', 'Sn', 'Hf']  
    desc_cols = ['W', 'EI', 'EA', 'μ']                      
    all_feat_cols = comp_cols + desc_cols                   
    label_col = ['d33(pC/N)']                               

    composition = data[comp_cols]
    descriptors = data[desc_cols]
    
    y = data[label_col].values  

    
    all_features = pd.concat([composition, descriptors], axis=1).values
    x_train, x_test, y_train, y_test = train_test_split(
        all_features, y, test_size=0.2, random_state=seed
    )

    
    composition_scaler = MinMaxScaler()
    x_train_comp = x_train[:, :7]
    x_test_comp = x_test[:, :7]
    x_train_comp_norm = composition_scaler.fit_transform(x_train_comp)
    x_test_comp_norm = composition_scaler.transform(x_test_comp)

    
    descriptors_scaler = MinMaxScaler()
    x_train_desc = x_train[:, 7:]
    x_test_desc = x_test[:, 7:]
    x_train_desc_norm = descriptors_scaler.fit_transform(x_train_desc)
    x_test_desc_norm = descriptors_scaler.transform(x_test_desc)

    
    joblib.dump(composition_scaler, f'{save_path}/composition_scaler.joblib')
    joblib.dump(descriptors_scaler, f'{save_path}/descriptors_scaler.joblib')

    
    x_train_norm = np.hstack([x_train_comp_norm, x_train_desc_norm])
    x_test_norm = np.hstack([x_test_comp_norm, x_test_desc_norm])
    
    x_comp_all = composition_scaler.transform(all_features[:, :7])
    x_desc_all = descriptors_scaler.transform(all_features[:, 7:])
    x_all_norm = np.hstack([x_comp_all, x_desc_all])

    
    train_df = pd.DataFrame(x_train_norm, columns=all_feat_cols)
    train_df[label_col[0]] = y_train  
    train_df.to_excel(f'{save_path}/train_set.xlsx', index=False, engine='openpyxl')

    
    test_df = pd.DataFrame(x_test_norm, columns=all_feat_cols)
    test_df[label_col[0]] = y_test  
    test_df.to_excel(f'{save_path}/test_set.xlsx', index=False, engine='openpyxl')

    
    all_df = pd.DataFrame(x_all_norm, columns=all_feat_cols)
    all_df[label_col[0]] = y
    all_df.to_excel(f'{save_path}/all_set.xlsx', index=False, engine='openpyxl')

    
    print(f"\n✅ 归一化数据已保存为Excel，路径：{save_path}")
    print(f"📊 训练集（特征+标签）：{train_df.shape} | 测试集：{test_df.shape} | 全量集：{all_df.shape}")
    print("📋 Excel包含列名，可直接用Excel打开查看/编辑")

    
    print("\n归一化数据形状验证：")
    print("全量特征：", x_all_norm.shape, "| 训练特征：", x_train_norm.shape, "| 测试特征：", x_test_norm.shape)
    print("标签形状：", y.shape)

    return (x_all_norm, y,          
            x_train_norm, x_test_norm,  
            y_train, y_test,            
            composition_scaler, descriptors_scaler)  


def load_normalized_data_excel(load_path='normalized_data_excel'):
    
    all_feat_cols = ['Ba', 'Ca', 'Sr', 'Ti', 'Zr', 'Sn', 'Hf', 'W', 'EI', 'EA', 'μ']
    label_col = 'd33(pC/N)'

   
    train_df = pd.read_excel(f'{load_path}/train_set.xlsx', engine='openpyxl')
    test_df = pd.read_excel(f'{load_path}/test_set.xlsx', engine='openpyxl')
    all_df = pd.read_excel(f'{load_path}/all_set.xlsx', engine='openpyxl')

    
    x_train_norm = train_df[all_feat_cols].values
    y_train = train_df[[label_col]].values
    x_test_norm = test_df[all_feat_cols].values
    y_test = test_df[[label_col]].values
    x_all_norm = all_df[all_feat_cols].values
    y_all = all_df[[label_col]].values

    
    composition_scaler = joblib.load(f'{load_path}/composition_scaler.joblib')
    descriptors_scaler = joblib.load(f'{load_path}/descriptors_scaler.joblib')

    
    print(f"\n✅ 已从{load_path}加载Excel格式归一化数据")
    print(f"📊 加载数据形状：训练特征{x_train_norm.shape} | 测试特征{x_test_norm.shape} | 全量特征{x_all_norm.shape}")

    return x_all_norm, y_all, x_train_norm, x_test_norm, y_train, y_test, composition_scaler, descriptors_scaler


if __name__ == '__main__':
    
    plt.close('all')
    
    data = pd.read_excel('data-1.xlsx', engine='openpyxl')
    print("原始数据形状：", data.shape)
    print("原始数据列名：", data.columns.tolist())

    
    x_all, y_all, x_train, x_test, y_train, y_test, _, _ = normalizing_data(data, seed=1)

   
    train_dataset = FeatureDataset(x_train, y_train)
    test_dataset = FeatureDataset(x_test, y_test)

    
    BATCH_SIZE = 32
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    
    print("\n训练集DataLoader单批次形状：")
    for batch_x, batch_y in train_dataloader:
        print("特征批次：", batch_x.shape, "| 标签批次：", batch_y.shape)
        break
    print("测试集DataLoader单批次形状：")
    for batch_x, batch_y in test_dataloader:
        print("特征批次：", batch_x.shape, "| 标签批次：", batch_y.shape)
        break


    print("\n训练特征前5行：\n", x_train[:5])
    print("训练标签前5行：\n", y_train[:5])

In [5]:
import datetime
import torch.utils.data as Data
import pandas as pd
import torch
import torch.nn.functional as F    
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from bayes_opt import BayesianOptimization
from sklearn import preprocessing
import os


class Net(nn.Module):  
    def __init__(self, n_feature = 11, n_hidden = 64, n_output = 1, w = 6):
        super(Net, self).__init__()    
        
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden) 
        nn.init.kaiming_normal_(self.hidden1.weight)
        
        self.hiddens = nn.ModuleList ([nn.Linear(n_hidden, n_hidden) for i in range(w)])                            
        for m in self.hiddens:
            nn.init.kaiming_normal_(m.weight)   
        
        self.predict = torch.nn.Linear(n_hidden, n_output)  
        nn.init.kaiming_normal_(self.predict.weight)

    def forward(self, x):  
        x = self.hidden1(x)
        
        
        x = F.relu(x)   
        
        for m in self.hiddens:
            x = m(x)
            
            x = F.relu(x) 
                   
        x = self.predict(x)
        
        return x
       

def train(net, num_epochs, batch_size, train_features, test_features, train_labels, test_labels,
          train_loader,
          optimizer):
    print ("\n=== train begin ===")
    
    train_ls, test_ls = [], []
    loss = MAPELoss() 
    for epoch in range(num_epochs):
        for x, y in train_loader:
            ls = loss(net(x).view(-1, 1), y.view(-1, 1))
            optimizer.zero_grad()
            ls.backward()
            optimizer.step()
        if epoch % 100 == 0:
            train_ls.append(loss(net(train_features).view(-1, 1), train_labels.view(-1, 1)).item())
            test_ls.append(loss(net(test_features).view(-1, 1), test_labels.view(-1, 1)).item())
            print ("epoch %d: train loss %f, test loss %f" % (epoch, train_ls[-1], test_ls[-1]))
        
    print ("=== train end ===")
    
def test(model, test_loader, set_name="Test set"):
    model.eval()
    test_loss = 0
    n = 0
    loss = MAPELoss() 
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss(output.view(-1, 1), target.view(-1, 1)).item()  
            n += 1

    test_loss /= n
    
    print(f'{set_name}: Average loss: {test_loss:.4f}')
    return test_loss
        
def plotCurve(x_vals, y_vals, 
                     x_label, y_label, 
                     x2_vals=None, y2_vals=None, 
                    legend=None,
                    figsize=(3.5, 2.5)):
            
            plt.xlabel(x_label)
            plt.ylabel(y_label)
            plt.plot(x_vals, y_vals)
            if x2_vals and y2_vals:
                plt.plot(x2_vals, y2_vals, linestyle=':')
            
            if legend:
                plt.legend(legend)

In [ ]:
import datetime
import torch.utils.data as Data
import pandas as pd
import torch
import torch.nn.functional as F    
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from bayes_opt import BayesianOptimization
import time
import os
import joblib  
from sklearn import preprocessing


class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()
    def forward(self, output, target):
        eps = 1e-8
        loss = torch.mean(torch.abs((target - output) / (target + eps)))
        return loss


class Net(nn.Module):  
    def __init__(self, n_feature = 11, n_hidden = 64, n_output = 1, w = 6):
        super(Net, self).__init__()    
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden) 
        nn.init.kaiming_normal_(self.hidden1.weight)
        self.hiddens = nn.ModuleList ([nn.Linear(n_hidden, n_hidden) for i in range(w)])                            
        for m in self.hiddens:
            nn.init.kaiming_normal_(m.weight)   
        self.predict = torch.nn.Linear(n_hidden, n_output)  
        nn.init.kaiming_normal_(self.predict.weight)
    def forward(self, x):  
        x = self.hidden1(x)
        x = F.relu(x)   
        for m in self.hiddens:
            x = m(x)
            x = F.relu(x) 
        x = self.predict(x)
        return x


def train(net, num_epochs, batch_size, train_features, test_features, train_labels, test_labels,
          train_loader, optimizer):
    print ("\n=== train begin ===")
    train_ls, test_ls = [], []
    loss = MAPELoss() 
    device = next(net.parameters()).device  
    for epoch in range(num_epochs):
        for x, y in train_loader:
            
            x, y = x.to(device), y.to(device)
            ls = loss(net(x).view(-1, 1), y.view(-1, 1))
            optimizer.zero_grad()
            ls.backward()
            optimizer.step()
        if epoch % 100 == 0:
            
            train_feat, train_lab = train_features.to(device), train_labels.to(device)
            test_feat, test_lab = test_features.to(device), test_labels.to(device)
            train_ls.append(loss(net(train_feat).view(-1, 1), train_lab.view(-1, 1)).item())
            test_ls.append(loss(net(test_feat).view(-1, 1), test_lab.view(-1, 1)).item())
            print (f"epoch {epoch}: train loss {train_ls[-1]:.4f}, test loss {test_ls[-1]:.4f}")
    print ("=== train end ===")


def test(model, test_loader, set_name="Test set"):
    model.eval()
    test_loss = 0
    n = 0
    loss = MAPELoss() 
    device = next(model.parameters()).device  
    with torch.no_grad():
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss(output.view(-1, 1), target.view(-1, 1)).item()  
            n += 1
    test_loss /= n
    print(f'{set_name}: Average loss: {test_loss:.4f}')
    return test_loss


def load_normalized_data_excel(load_path='normalized_data_excel'):
    
    
    all_feat_cols = ['Ba', 'Ca', 'Sr', 'Ti', 'Zr', 'Sn', 'Hf', 'W', 'EI', 'EA', 'μ']
    label_col = 'd33(pC/N)'

    
    train_df = pd.read_excel(f'{load_path}/train_set.xlsx', engine='openpyxl')
    test_df = pd.read_excel(f'{load_path}/test_set.xlsx', engine='openpyxl')

    
    train_features = torch.FloatTensor(train_df[all_feat_cols].values)
    train_labels = torch.FloatTensor(train_df[[label_col]].values)
    test_features = torch.FloatTensor(test_df[all_feat_cols].values)
    test_labels = torch.FloatTensor(test_df[[label_col]].values)

    print(f"\n✅ 成功加载Excel训练/测试集")
    print(f"训练集：特征{train_features.shape} | 标签{train_labels.shape}")
    print(f"测试集：特征{test_features.shape} | 标签{test_labels.shape}")

    return train_features, train_labels, test_features, test_labels


starttime = datetime.datetime.now()

output_dir = 'NNBayesian'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


train_features, train_labels, test_features, test_labels = load_normalized_data_excel()


def train_model(batch_size, lr, module__n_hidden, module__w):
    module__n_hidden = int(module__n_hidden)  
    module__w = int(module__w)  
    batch_size = int(batch_size)

    
    train_dataset = Data.TensorDataset(train_features, train_labels)
    test_dataset = Data.TensorDataset(test_features, test_labels)
    
    train_loader = Data.DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = Data.DataLoader(test_dataset, batch_size, shuffle=False)

    
    net = Net(n_feature=11, n_hidden=module__n_hidden, n_output=1, w=module__w)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
    net.to(device)  

    
    n_epochs = 1000
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.0001)

    
    train(net, n_epochs, batch_size, train_features, test_features, train_labels, test_labels, train_loader, optimizer)
    
    
    train_loss = test(net, train_loader, set_name="Training set")  
    test_loss = test(net, test_loader, set_name="Test set")  

    
    r = -np.abs(train_loss - test_loss)
    return -test_loss


bounds = {'lr': (0.0001, 0.001), 
          'batch_size': (32, 64), 
          'module__n_hidden': (64, 256), 
          'module__w': (2, 10)}


bo_optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)


bo_optimizer.maximize(init_points=100, n_iter=150)  


result_list = []  
for res in bo_optimizer.res:
    result_list.append(pd.DataFrame({'target': [res['target']],
                                     'batch_size': [res['params']['batch_size']],
                                     'lr': [res['params']['lr']],
                                     'module__n_hidden': [res['params']['module__n_hidden']],
                                     'module__w': [res['params']['module__w']]}))

table = pd.concat(result_list, ignore_index=True)


table = pd.concat([table, pd.DataFrame({'target': [bo_optimizer.max['target']],
                                        'batch_size': [bo_optimizer.max['params']['batch_size']],
                                        'lr': [bo_optimizer.max['params']['lr']],
                                        'module__n_hidden': [bo_optimizer.max['params']['module__n_hidden']],
                                        'module__w': [bo_optimizer.max['params']['module__w']]})], 
                                        ignore_index=True)


model_name = 'd33_inference_NN_{}'.format(datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
file_name = os.path.join(output_dir, '{}.xlsx'.format(model_name))


endtime = datetime.datetime.now()
Rtime = endtime - starttime
print(f"\n总运行时间：{Rtime}")


table.to_excel(file_name, index=False, engine='openpyxl')  
print("保存调参结果至: ", file_name)
print("\n最优超参数：")
print(bo_optimizer.max)

In [ ]:
model_name = 'd33_inference_NN'
file_name = '{}.xlsx'.format(model_name)
endtime = datetime.datetime.now()
Rtime = endtime - starttime
print(Rtime)
table.to_excel(file_name)
print(table)

In [ ]:
import datetime
import torch.utils.data as Data
import pandas as pd
import torch
import torch.nn.functional as F   
import matplotlib.pyplot as plt
import numpy as np
import random  
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import time
import os
import seaborn as sns
from sklearn.metrics import r2_score
import joblib  
from sklearn.preprocessing import MinMaxScaler


class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()
    def forward(self, output, target):
        eps = 1e-8
        loss = torch.mean(torch.abs((target - output) / (target + eps)))
        return loss


def load_normalized_data_excel(load_path='normalized_data_excel'):
    all_feat_cols = ['Ba', 'Ca', 'Sr', 'Ti', 'Zr', 'Sn', 'Hf', 'W', 'EI', 'EA', 'μ']
    label_col = 'd33(pC/N)'
    
    train_df = pd.read_excel(f'{load_path}/train_set.xlsx', engine='openpyxl')
    test_df = pd.read_excel(f'{load_path}/test_set.xlsx', engine='openpyxl')
    all_df = pd.read_excel(f'{load_path}/all_set.xlsx', engine='openpyxl')
    
    x_train = torch.FloatTensor(train_df[all_feat_cols].values)
    y_train = torch.FloatTensor(train_df[[label_col]].values)
    x_test = torch.FloatTensor(test_df[all_feat_cols].values)
    y_test = torch.FloatTensor(test_df[[label_col]].values)
    x_all = torch.FloatTensor(all_df[all_feat_cols].values)
    y_all = torch.FloatTensor(all_df[[label_col]].values)
    
    comp_scaler = joblib.load(f'{load_path}/composition_scaler.joblib')
    desc_scaler = joblib.load(f'{load_path}/descriptors_scaler.joblib')
    print("✅ 成功加载归一化Excel数据（Tensor格式）")
    print(f"全量：{x_all.shape} | 训练：{x_train.shape} | 测试：{x_test.shape}")
    return x_all, y_all, x_train, x_test, y_train, y_test


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_random_seed(1)

folder_dir = 'Results/STU_NN_BO(100+150)_1'
os.makedirs(folder_dir, exist_ok=True)  

folder_dir_figures = os.path.join(folder_dir, 'Figures')
os.makedirs(folder_dir_figures, exist_ok=True)  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 训练设备：{device}")

plt.close('all')

target = pd.read_excel('d33_inference_NN.xlsx', engine='openpyxl')

starttime = datetime.datetime.now()


results_df = pd.DataFrame(columns=[
    'Iteration', 'Seed', 'target', 'R2_Score_test', 'Figure_Path_test',
    'R2_Score_train', 'Figure_Path_train', 'R2_Score_all', 'Figure_Path_all',
    'Final_Train_Loss', 'Final_Test_Loss'
])


x_all, y_all, train_features, test_features, train_labels, test_labels = load_normalized_data_excel()

x_all, y_all = x_all.to(device), y_all.to(device)
train_features, train_labels = train_features.to(device), train_labels.to(device)
test_features, test_labels = test_features.to(device), test_labels.to(device)

train_df = pd.DataFrame(train_features.cpu().numpy())
test_df = pd.DataFrame(test_features.cpu().numpy())
train_file_path = os.path.join(folder_dir, 'train_features.xlsx')  
test_file_path = os.path.join(folder_dir, 'test_features.xlsx')    
train_df.to_excel(train_file_path, index=True, engine='openpyxl')  
test_df.to_excel(test_file_path, index=True, engine='openpyxl')   
print(f"📄 训练/测试特征已保存至 {folder_dir}")


for i in range(0, 251):  
    
    for j in range(1, 2):  
        set_random_seed(1)
        print(f"\n===== 开始训练：第{i}轮调参结果 | 种子{j} =====")

        
        tg = target.at[i, 'target']
        lr = target.at[i, 'lr'] 
        module__n_hidden = target.at[i, 'module__n_hidden']
        module__w = target.at[i, 'module__w']
        batch_size = target.at[i, 'batch_size']
        
        module__n_hidden = int(module__n_hidden)
        module__w = int(module__w)
        batch_size = int(batch_size)

        
        train_dataset = Data.TensorDataset(train_features, train_labels)
        test_dataset = Data.TensorDataset(test_features, test_labels) 
        train_loader = Data.DataLoader(train_dataset, batch_size, shuffle=True)
        test_loader = Data.DataLoader(test_dataset, batch_size, shuffle=False) 

        
        class Net(nn.Module):  
            def __init__(self, n_feature=11, n_hidden=module__n_hidden, n_output=1, w=module__w):
                super(Net, self).__init__()   
                self.hidden1 = nn.Linear(n_feature, n_hidden) 
                nn.init.kaiming_normal_(self.hidden1.weight)
                self.hiddens = nn.ModuleList([nn.Linear(n_hidden, n_hidden) for _ in range(w)])                            
                for m in self.hiddens:
                    nn.init.kaiming_normal_(m.weight)   
                self.dropout = nn.Dropout(p=0.01)  
                self.predict = nn.Linear(n_hidden, n_output) 
                nn.init.kaiming_normal_(self.predict.weight)

            def forward(self, x): 
                x = self.hidden1(x)
                x = F.relu(x)   
                x = self.dropout(x)
                for m in self.hiddens:
                    x = m(x)
                    x = F.relu(x) 
                    x = self.dropout(x)          
                x = self.predict(x)
                return x

        
        net = Net().to(device)
        
        train_ls, test_ls = [], []
        loss = MAPELoss() 
        n_epochs = 1000
        optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.0001)
        
        
        for epoch in range(n_epochs):
            net.train()  
            for x, y in train_loader:
                x, y = x.to(device), y.to(device)  
                ls = loss(net(x).view(-1, 1), y.view(-1, 1))
                optimizer.zero_grad()
                ls.backward()
                optimizer.step()
            
            net.eval()  
            with torch.no_grad():  
                train_loss = loss(net(train_features).view(-1, 1), train_labels.view(-1, 1)).item()
                test_loss = loss(net(test_features).view(-1, 1), test_labels.view(-1, 1)).item()
                train_ls.append(train_loss)
                test_ls.append(test_loss)

        
        loss_data = pd.DataFrame({'Epoch': range(1, n_epochs + 1), 'Train Loss': train_ls, 'Test Loss': test_ls})
        loss_file_path = os.path.join(folder_dir, f'loss_data_{i}_seed_{j}.xlsx')
        loss_data.to_excel(loss_file_path, index=False, engine='openpyxl')

        
        plt.figure(figsize=(8, 6))
        plt.plot(range(1, n_epochs + 1), train_ls, label="Train Loss", color='blue')
        plt.plot(range(1, n_epochs + 1), test_ls, label="Test Loss", color='orange')
        plt.xlabel('Epoch'), plt.ylabel('MAPE Loss'), plt.legend()
        plt.title(f'Loss Curve - Iter {i} Seed {j}')
        plt.text(60, max(train_ls)*0.7, f'Target={tg:.4f}', fontdict={'size': 12, 'color':  'red'})
        fig_name_1 = os.path.join(folder_dir_figures, f'{i}-seed_{j}_loss.png')
        plt.savefig(fig_name_1, format='png', dpi=300, bbox_inches='tight')
        plt.close()  

       
        net.eval()
        with torch.no_grad():
            predict_test = net(test_features).cpu().data.numpy()
            predict_train = net(train_features).cpu().data.numpy()
            predict_all = net(x_all).cpu().data.numpy()
        
        y_test_np = test_labels.cpu().data.numpy()
        y_train_np = train_labels.cpu().data.numpy()
        y_all_np = y_all.cpu().data.numpy()

        
        pd.DataFrame({
            'Predicted': predict_train.flatten(), 
            'Actual': y_train_np.flatten()
        }).to_excel(os.path.join(folder_dir, f'predictions_train_{i}_seed_{j}.xlsx'), index=False, engine='openpyxl')
        pd.DataFrame({
            'Predicted': predict_test.flatten(), 
            'Actual': y_test_np.flatten()
        }).to_excel(os.path.join(folder_dir, f'predictions_test_{i}_seed_{j}.xlsx'), index=False, engine='openpyxl')
        pd.DataFrame({
            'Predicted': predict_all.flatten(), 
            'Actual': y_all_np.flatten()
        }).to_excel(os.path.join(folder_dir, f'predictions_all_{i}_seed_{j}.xlsx'), index=False, engine='openpyxl')

        
        fig_name_2_test = os.path.join(folder_dir_figures, f'{i}-seed_{j}_test.png')
        plt.figure(figsize=(6, 6))
        sns.regplot(x=predict_test, y=y_test_np, color='red', line_kws={'color':'black'})
        plt.xlabel('Predicted d33'), plt.ylabel('Actual d33'), plt.title(f'Test Set - Iter {i} Seed {j}')
        r2_test = r2_score(y_test_np, predict_test)
        plt.text(min(predict_test), max(y_test_np), f'R2={r2_test:.4f}', color='red', fontsize=12)
        plt.savefig(fig_name_2_test, format='png', dpi=300, bbox_inches='tight')
        plt.close()

        
        fig_name_2_train = os.path.join(folder_dir_figures, f'{i}-seed_{j}_train.png')
        plt.figure(figsize=(6, 6))
        sns.regplot(x=predict_train, y=y_train_np, color='blue', line_kws={'color':'black'})
        plt.xlabel('Predicted d33'), plt.ylabel('Actual d33'), plt.title(f'Train Set - Iter {i} Seed {j}')
        r2_train = r2_score(y_train_np, predict_train)
        plt.text(min(predict_train), max(y_train_np), f'R2={r2_train:.4f}', color='blue', fontsize=12)
        plt.savefig(fig_name_2_train, format='png', dpi=300, bbox_inches='tight')
        plt.close()

        
        fig_name_2_all = os.path.join(folder_dir_figures, f'{i}-seed_{j}_all.png')
        plt.figure(figsize=(6, 6))
        sns.regplot(x=predict_train, y=y_train_np, color='blue', label="Train", line_kws={'color':'blue'})
        sns.regplot(x=predict_test, y=y_test_np, color='red', label="Test", line_kws={'color':'red'})
        plt.xlabel('Predicted d33'), plt.ylabel('Actual d33'), plt.title(f'All Set - Iter {i} Seed {j}')
        r2_all = r2_score(y_all_np, predict_all)
        plt.text(min(predict_all), max(y_all_np), f'R2={r2_all:.4f}', color='green', fontsize=12)
        plt.legend(), plt.axis('equal')
        plt.savefig(fig_name_2_all, format='png', dpi=300, bbox_inches='tight')
        plt.close()

        
        final_train_loss = train_ls[-1]  
        final_test_loss = test_ls[-1]  

        
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Iteration': i,
            'Seed': j,
            'target': tg,
            'R2_Score_test': r2_test,
            'Figure_Path_test': fig_name_2_test,
            'R2_Score_train': r2_train,
            'Figure_Path_train': fig_name_2_train,
            'R2_Score_all': r2_all,
            'Figure_Path_all': fig_name_2_all,
            'Final_Train_Loss': final_train_loss,
            'Final_Test_Loss': final_test_loss   
        }])], ignore_index=True)

        
        net_name = os.path.join(folder_dir, f'{i}-seed_{j}.pt')
        torch.save(net.state_dict(), net_name)
        print(f"✅ 第{i}轮训练完成：模型保存至 {net_name}")


results_df.to_csv(os.path.join(folder_dir, 'results_summary_NN.csv'), index=False, encoding='utf-8')

endtime = datetime.datetime.now()
Rtime = endtime - starttime
print(f"\n===== 所有训练完成 =====")
print(f"总训练耗时：{Rtime}")
print(f"所有结果已保存至：{folder_dir}")